# Introduction to trackintel

This tutorial will show you the most important functionalities of trackintel.

## Model

Trackintel is based on a hierachical data model. 
![title](../docs/assets/hierarchy.png)

The details are explained in readme.md.

To get started, import the framework using the following command:

In [ ]:
import trackintel as ti
import geopandas as gpd
import pandas as pd
import os

As a first step, the data that should be processed has to be importet. In this example we use a stored GEOJSON file, that contains raw positionfixes.

In [ ]:
gdf = gpd.read_file('./data/geolife.geojson')
pfs = ti.io.from_geopandas.positionfixes_from_gpd(gdf)

The geodataframe (gdf) imported by the GeoPandas read_file function gets validated by calling 